<a href="https://colab.research.google.com/github/Mahtab14H/BDDA-all/blob/main/sentimentAnalysis_for_DLS%201.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# LAst amended: 01st Feb, 2024
# Objective: To prepare csv file from tweets data
#            for experimenting in deeplearning studio (DLS)
#
# My Github reference:
#    https://github.com/harnalashok/deeplearning-sequences/blob/main/sentimentAnalysis_for_DLS.ipynb
#
# Ref: https://community.deepcognition.ai/t/text-processing-and-tutorial-video-for-uploading-text-dataset/238

## How to?<br>
> 1.0 Your csv file must have at the least two columns. If there are more columns, no problem.<br>
> 2.0 The tweets or text column must have the header name as <b>'text'</b> and the class column should have the header name as <b>'label'</b>  
> 3.0 Upload csv file in your gdrive<br>
> 4.0 After the process is complete, the saved file will be in the folder '/content'. You can download the processed file directly from this folder. Right click on it and click <b>Download</b>.

## Call libraries

In [2]:
# 1.0
import tensorflow
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import csv

In [3]:
# 1.1
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


## Read data
Change here your csv file location appropriately.

In [4]:
# 2.0 Change this path as per your file location in gdrive:
#datapath = "/content/train.csv"
#datapath = "/gdrive/MyDrive/Colab_data_files/disaster_tweets/socialmedia_relevant_cols.csv"
datapath = "/gdrive/MyDrive/train.csv"

In [5]:
# 2.1 If problem in reading try different encodings
#      For other encodings, see: https://stackoverflow.com/a/18172249/3282777#

data = pd.read_csv(datapath, encoding = "ISO-8859-1")
data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
# 2.2 How many classes does this data have?

#data['class_label'].value_counts()  # Three: 0,1,2
data['target'].value_counts() #here colm name class label changed to target since in our file colm is target

0    4342
1    3271
Name: target, dtype: int64

In [7]:
# 2.3 If three, then remove the class with label 2:

#data = data.loc[data['class_label'] != 2, :]
data = data.loc[data['target'] != 2, :]

In [8]:
# 2.4 Check again now:

#data['class_label'].value_counts()
data['target'].value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [9]:
# 2.5 Target should not have any NULL values:

#data['class_label'].isnull().sum()
data['target'].isnull().sum()

0

In [10]:
#text_file = open("reviews.txt", "r")
#lines = text_file.readlines()

In [11]:
#2.6 Get rows from data:

lines = data['text']

In [12]:
lines #just to see it is added

0       Our Deeds are the Reason of this #earthquake M...
1                  Forest fire near La Ronge Sask. Canada
2       All residents asked to 'shelter in place' are ...
3       13,000 people receive #wildfires evacuation or...
4       Just got sent this photo from Ruby #Alaska as ...
                              ...                        
7608    Two giant cranes holding a bridge collapse int...
7609    @aria_ahrary @TheTawniest The out of control w...
7610    M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...
7611    Police investigating after an e-bike collided ...
7612    The Latest: More Homes Razed by Northern Calif...
Name: text, Length: 7613, dtype: object

## Tokenize data

In [13]:
# 3.0 Select relevant parameters:

maxlen = 500                  # Maximum length of reviews. A review greater than maxlen
                              #  will be truncated
max_words = 10000             # We will only consider the top max_words in the dataset

In [14]:
# 3.1 Instantiate Tokenizer class
tokenizer = Tokenizer(num_words=max_words)

In [15]:
# 3.2 Fit it on text

tokenizer.fit_on_texts(lines)   # tokenizer.index_word

In [16]:
# 3.3 Transform text, tweet by tweet, as a list of numbers:

sequences = tokenizer.texts_to_sequences(lines)
print(sequences[:3])  # Print top 3-comments/tweets

[[119, 4633, 24, 4, 868, 8, 21, 263, 138, 1619, 4634, 89, 40], [189, 45, 229, 799, 6954, 6955, 1404], [40, 1751, 1620, 7, 6956, 6, 6957, 24, 136, 6958, 20, 1752, 39, 441, 256, 57, 2158, 6, 714, 1405, 24, 1106]]


In [17]:
# 3.4 How many tweets have been read:

len(sequences)


7613

## Pad all sequences to same length

In [18]:
# 4.0 Trasnform sequences to sameLengthSequences :
sameLengthSequences = pad_sequences(sequences, maxlen=maxlen)

In [19]:
# 4.1 Join every number in a sequence using semicolon:
#     The three sequences: [[0,0,23,45], [89,76,33,44],[49,98,34,22]]
#     become:              [[0;0;23;45], [89;76;33;44],[49;98;34;22]]

sequencesToStrings = []
for row in sameLengthSequences:
    sequencesToStrings.append(';'.join(str(col) for col in row))

## Save file

In [20]:
# 4.2 Our csv file:

csvfile = "processed.csv"

In [ ]:
#with open(csvfile, "w") as output:
#    writer = csv.writer(output, lineterminator='\n')
#    for val in sequencesToStrings:
#        writer.writerow([val])

In [21]:
# 4.3 Create a blank dataframe
s = pd.DataFrame()

In [22]:
# 4.4 One column is text column
s['text'] = sequencesToStrings

In [23]:
# 4.5 The other is label column:
s['Label'] = data['target'].values


In [24]:
# 4.6 Save the file appropriately:
#     It is saved to /content folder:

s.to_csv(csvfile, index = False)

## Steps to take now:

DLS for tabular data
====================
 USe Chrome Browser

 Steps:<br>
 1.0 Rename your processed .csv file as 'train.csv'  
 2.0 Place your train.csv in an empty folder  
 3.0 Name of this folder will be the name of your dataset in the DLS. So rename the folder, if needed.  
 4.0 zip the folder  
 5.0 Upload the zipped folder in DLS with  DLS Native
     While uploading select DLS Native dataset option<br>
 6.0 Create a project and develop model as usual. To start with you can use the same model as in DLS for IMDB dataset.IMDB dataset is very large (15000 training samples) as ours of just 2000 samples. Try to make this sample alsolarge.




In [ ]:
########## DONE ############